In [243]:
#Created: 15.02.2016
#Last modified: 18.02.2016
#               29.02.2016 add Corr and Dis cols
#               25.05.2016 updated input file and created truncated file for testing
#               24.05.2016 tested reading in file produced in birdhouse analogs process
#               05.09.2016 format test file for dc chart brush problem
#               09.09.2016 added test file that spans two months to test date handling

In [1]:
import numpy as np
import pandas as pd
import collections
import os

In [2]:
num_analogues = 20 #number of analogues searched for
num_cols = 3 #dateAnlg, Dis, Corr

In [3]:
#To read file produced in birdhouse
dfS = pd.DataFrame()
#analogs = 'http://birdhouse-lsce.extra.cea.fr:8090/wpsoutputs/flyingpigeon/output_txt-0797016c-378e-11e6-91dd-41d8cd554993.txt'
#analogs = 'http://localhost:8090/wpsoutputs/flyingpigeon/analogs-c26ba398-711e-11e6-8d5a-fb3204a8dad1.txt'

#Analysis period > 1 year
#analogs = 'http://birdhouse-lsce.extra.cea.fr:8090/wpsoutputs/flyingpigeon/analogs-5ee81d98-6fc9-11e6-a296-4d3fe2c8a29e.txt'
analogs = 'http://birdhouse-lsce.extra.cea.fr:8090/wpsoutputs/flyingpigeon/analogs-73cd782c-74f4-11e6-bf5f-f73f2a3d7e35.txt'

#Analysis period spans two months (30.06.2013 to 02.07.2013)
#analogs = 'http://localhost:8090/wpsoutputs/flyingpigeon/analogs-7aafc4ec-74ea-11e6-b77d-eb5606a7fe9f.txt'

dfS = pd.read_csv(analogs, delimiter=r"\s+", index_col=0)
dfS.head()

,date.an1,date.an2,date.an3,date.an4,date.an5,date.an6,date.an7,date.an8,date.an9,date.an10,...,cor11,cor12,cor13,cor14,cor15,cor16,cor17,cor18,cor19,cor20
date,,,,,,,,,,,,,,,,,,,,,
20130101,20131228,20141215,20131229,20140104,20141219,20141216,20141214,20140113,20131207,20131231,...,0.086539,0.186083,0.198691,0.232033,-0.036342,0.228655,0.105518,0.066888,0.142485,0.157468
20130102,20131208,20131211,20141231,20131229,20131217,20141215,20131207,20141230,20140123,20131216,...,0.231459,0.004410,0.375896,0.411108,-0.078786,0.100821,0.259724,-0.016893,0.287747,0.048639
20130103,20141230,20141231,20131208,20131211,20131217,20131212,20131229,20131207,20141220,20131209,...,0.293782,0.349906,0.217645,0.440678,0.151341,0.113716,0.283423,-0.003058,0.270340,0.065022
20130104,20141230,20141231,20141229,20131208,20131207,20131211,20131212,20131213,20131209,20131217,...,0.340047,0.393674,0.252089,0.178680,0.103945,0.189505,0.214643,0.419906,0.291117,0.135634
20130105,20141229,20141230,20141228,20141231,20131208,20131207,20140123,20131211,20131210,20131209,...,0.217104,0.320012,0.177326,0.590540,0.043217,0.106392,0.059386,0.492298,0.497847,0.108963


In [4]:
# Create dataframe from analogues file
# Latest file: /home/estimr1/yiou/estimr1/NCEP/slpano-NA.analog30rms.1d.all1948-2016.dat
# 2015: /home/estimr1/yiou/estimr1/NCEP/slpano-NA.analog30rms.1d.all1948-2015.dat
# Note that last day in 2015 file is 20151225 (and NOT 20151231) because that is the last sunday of Dec 2015.

#dfS = pd.DataFrame()
#dfS = pd.read_csv('/home/estimr1/yiou/estimr1/NCEP/slpano-NA.analog30rms.1d.all1948-2016.dat', delimiter=' ', index_col=0)
#dfS.set_index('Date', inplace=True)

#dfS.head(5)

#dfS.index.name

In [5]:
dfS.tail(5)

,date.an1,date.an2,date.an3,date.an4,date.an5,date.an6,date.an7,date.an8,date.an9,date.an10,...,cor11,cor12,cor13,cor14,cor15,cor16,cor17,cor18,cor19,cor20
date,,,,,,,,,,,,,,,,,,,,,
20141227,20130114,20130123,20130115,20130113,20130122,20130110,20130106,20131207,20130124,20130108,...,0.362514,0.429099,0.340225,0.398280,0.302674,0.492298,0.114743,0.437922,0.357190,0.173998
20141228,20130106,20130105,20130117,20130116,20130108,20130110,20130107,20130114,20130124,20130115,...,0.499869,0.337189,0.221587,0.482423,0.485476,0.393674,0.325742,0.322192,0.164484,0.407533
20141229,20130105,20130106,20130107,20130108,20130104,20130117,20140123,20130124,20130109,20130116,...,0.604046,0.288543,0.331964,0.324073,0.370352,0.366022,0.430320,0.214364,0.331960,0.261593
20141230,20130106,20130108,20130107,20130105,20130104,20130109,20130125,20131211,20131208,20140123,...,0.417316,0.507642,0.444588,0.471250,0.345777,0.241390,0.163596,0.319384,0.194016,0.058472
20141231,20131217,20130104,20130108,20130125,20140124,20130109,20130105,20131213,20131208,20131212,...,0.493749,0.499308,0.430335,0.507305,0.384801,0.500900,0.430752,0.418686,0.300504,0.307457


In [6]:
dfS.index

Int64Index([20130101, 20130102, 20130103, 20130104, 20130105, 20130106, 20130107, 20130108, 20130109, 20130110, 20130111, 20130112, 20130113, 20130114, 20130115, 20130116, 20130117, 20130118, 20130119, 20130120, 20130121, 20130122, 20130123, 20130124, 20130125, 20130126, 20130127, 20130128, 20130129, 20130130, 20130131, 20130201, 20130202, 20130203, 20130204, 20130205, 20130206, 20130207, 20130208, 20130209, 20130210, 20130211, 20130212, 20130213, 20130214, 20130215, 20130216, 20130217, 20130218, 20130219, 20130220, 20130221, 20130222, 20130223, 20130224, 20130225, 20130226, 20130227, 20130228, 20130301, 20130302, 20130303, 20130304, 20130305, 20130306, 20130307, 20130308, 20130309, 20130310, 20130311, 20130312, 20130313, 20130314, 20130315, 20130316, 20130317, 20130318, 20130319, 20130320, 20130321, 20130322, 20130323, 20130324, 20130325, 20130326, 20130327, 20130328, 20130329, 20130330, 20130331, 20130401, 20130402, 20130403, 20130404, 20130405, 20130406, 20130407, 20130408, 20130409

In [7]:
len(dfS.index)

730

In [8]:
#Chop off 2016 rows NO!!! there will still be 2016 dates found as analogues, which messes up js script
#dfS = dfS[dfS.index <= 20151231]

In [9]:
#SAVE DATE INDEX
#DF['id2'] = DF.index
dfS['dateRef'] = dfS.index
dfS['dateRef'].head(10)

date
20130101    20130101
20130102    20130102
20130103    20130103
20130104    20130104
20130105    20130105
20130106    20130106
20130107    20130107
20130108    20130108
20130109    20130109
20130110    20130110
Name: dateRef, dtype: int64

In [10]:
#TRANSPOSE ANLG DATE COLS INTO SINGLE COL

#Define temporary df containing only anlg dates cols
df_anlg = dfS.iloc[:, 0:20]

#remove index name before transpose (stack)
df_anlg.index.name = ""

#transpose (stack)into single col
dateStack = df_anlg.stack()
#dateStack

In [11]:
#TRANSPOSE DIS COLS INTO SINGLE COL

#Define temporary df containing only dis cols
df_dis = dfS.iloc[:, 20:40]

#remove index name before transpose (stack)
df_dis.index.name = ""

#transpose (stack) into single col and take absolute value
disStack = df_dis.stack().abs()
disStack.head(5)

20130101  dis1     913.126
          dis2     927.622
          dis3     935.474
          dis4    1050.258
          dis5    1053.329
dtype: float64

In [12]:
#TRANSPOSE CORR COLS INTO SINGLE COL

#Define temporary df containing only corr cols
df_corr = dfS.iloc[:, 40:60]

#remove index name before transpose (stack)
df_corr.index.name = ""

#transpose (stack) into single col
corrStack = df_corr.stack()
corrStack.head(5)

20130101  cor1    0.378995
          cor2    0.520871
          cor3    0.300302
          cor4    0.319006
          cor5    0.357050
dtype: float64

In [13]:
#BUILD NEW DF IN CORRECT FORMAT

#CREATE DF OF CORRECT DIMENSIONS (n x num_cols)
df_all = dfS.iloc[:, 0:num_cols]
#rename cols
df_all.columns = ['dateAnlg', 'Dis', 'Corr']

In [14]:
#dfS.index.tolist()
df_all.head(5)

,dateAnlg,Dis,Corr
,,,
20130101,20131228,20141215,20131229
20130102,20131208,20131211,20141231
20130103,20141230,20141231,20131208
20130104,20141230,20141231,20141229
20130105,20141229,20141230,20141228


In [15]:
df_all = df_all.set_index(dfS['dateRef'] )
df_all.head(10)

,dateAnlg,Dis,Corr
dateRef,,,
20130101,20131228,20141215,20131229
20130102,20131208,20131211,20141231
20130103,20141230,20141231,20131208
20130104,20141230,20141231,20141229
20130105,20141229,20141230,20141228
20130106,20141230,20141229,20141228
20130107,20141230,20141229,20140123
20130108,20141230,20141231,20141229
20130109,20141230,20141231,20140111


In [16]:
#REPLICATE INDEX COL 20 TIMES PER ROW

#http://stackoverflow.com/questions/26777832/replicating-rows-in-a-pandas-data-frame-by-a-column-value
#df2 = df.loc[np.repeat(df.index.values,df.n)]
df_all = df_all.loc[np.repeat(df_all.index.values,num_analogues)]

In [17]:
#REPLACE COLS WITH STACKED ANLG DATES, DIS AND CORR
df_all.iloc[:,0] = list(dateStack)
df_all.iloc[:,1] = list(disStack)
df_all.iloc[:,2] = list(corrStack)
df_all.head(5)

,dateAnlg,Dis,Corr
dateRef,,,
20130101,20131228,913.126,0.378995
20130101,20141215,927.622,0.520871
20130101,20131229,935.474,0.300302
20130101,20140104,1050.258,0.319006
20130101,20141219,1053.329,0.357050


In [18]:
df_all.shape

(14600, 3)

In [19]:
df_all.tail(5)

,dateAnlg,Dis,Corr
dateRef,,,
20141231,20130103,989.458,0.500900
20141231,20130102,1004.926,0.430752
20141231,20130106,1007.554,0.418686
20141231,20131220,1037.976,0.300504
20141231,20131216,1068.155,0.307457


In [33]:
#dfS_select = df_all[df_all.index <= 19600101]

In [20]:
#SAVE TO CSV FILE

#df_all.to_csv('analogues_19480101_20151225.json', float_format='%.2f', sep='\t')
#df_all.to_csv('test2.json', float_format='%.2f', sep='\t')
#df_all.to_csv('test_gt1yr.json', float_format='%.2f', sep='\t')
df_all.to_csv('test_gt1yr_birdhouse.json', float_format='%.2f', sep='\t')
#df_all.to_csv('test_span2months.json', float_format='%.2f', sep='\t')

#df_all.to_csv('analogues_19480101_20160520.json', float_format='%.2f', sep='\t')

#dfS_select.to_csv('analogues_select.json', float_format='%.2f', sep='\t')

In [77]:
pwd

u'/homel/cnangini/PROJECTS/analogues'

In [86]:
print df_all['Corr'].min(), df_all['Corr'].max()

-0.0773818303236 0.952985818969


In [87]:
print df_all['Dis'].min(), df_all['Dis'].max()

284.431297359 1369.41118104


In [31]:
#check 1970-1979
df_check =  df_all[ (df_all['dateAnlg'] >= 19700101) & (df_all['dateAnlg'] <= 19791231) ]


#((df['column1']=='banana') & (df['colour']=='green'))

In [32]:
df_check.shape

(75935, 3)

In [33]:
df_check =  df_all[ (df_all['dateAnlg'] >= 20100101) & (df_all['dateAnlg'] <= 20191231) ]

In [34]:
df_check.shape

(44554, 3)

In [35]:
#cut file for testing purposes
df_select = df_all.iloc[19000:19100,:]

In [36]:
df_select.shape

(100, 3)

In [37]:
df_select.tail(10)

,dateAnlg,Dis,Corr
dateRef,,,
19500812,19840828,449.988840,0.546204
19500812,19590909,453.044659,0.612506
19500812,20030819,453.638210,0.559760
19500812,19970801,457.039066,0.597291
19500812,19980815,459.252587,0.534387
19500812,19990829,466.308358,0.580896
19500812,19510820,466.860763,0.727749
19500812,20130816,467.150673,0.623782
19500812,19990717,471.338490,0.571134


In [38]:
df_chunk = df_all.iloc[149000:149100,:]
df_chunk2 = df_all.iloc[106000:106100,:]

In [39]:
df_select = df_select.append(df_chunk)
df_select = df_select.append(df_chunk2)
#df_select = pd.concat([df_select,df_chunk])

In [40]:
df_select.shape

(300, 3)

In [41]:
#SAVE SELECTION TO TSV FILE

#df_select.to_csv('analogues_reformat_all_select.tsv', float_format='%.2f', sep='\t')

In [42]:
print df_all['Corr'].min(), df_all['Corr'].max()

-0.0773818303236 0.952985818969


In [43]:
print df_all['Dis'].min(), df_all['Dis'].max()

284.431297359 1369.41118104


In [44]:
df_select['Corr'].max()
df_select['Corr'].min()
print df_select['Corr'].min(), df_select['Corr'].max()

0.124791552993 0.821019151951


In [45]:
df_select['Dis'].max()
df_select['Dis'].min()
print df_select['Dis'].min(), df_select['Dis'].max()

350.217929418 696.281040107
